# Python для анализа данных

*Алла Тамбовцева, НИУ ВШЭ*

### Работа с txt-файлами. Предобработка текста.

Сначала мы посмотрим, как считывать текст из txt-файлов. Это необходимое умение, поскольку не всегда набор текстов сохранен в виде готовой таблицы и выгружен в csv-файл. Часто приходится иметь дело с множеством txt-файлов, которые просто лежат в одной папке. 

Откроем txt-файл, в котором сохранено описание фильма «Господин оформитель» из Википедии.

In [1]:
f = open('mytext.txt', 'r', encoding = 'UTF-8')

Для открытия файла используется функция `open()`. Так как мы открываем файл только для чтения, мы выставляем флаг (аргумент) `r` (от *read*). Если нужно открыть файл сразу для всего (чтение, изменение, сохранение), то можно выставить флаг `a` (от *all*). При открытии файла лучше сразу указывать его кодировку, особенно если файл не на латинице, здесь это `UTF-8`.

Чтобы считать строки в файле, понадобится метод `.readlines()`:

In [2]:
f.readlines()

У этого метода есть одна особенность, не очень приятная, если к ней не привыкнуть ‒ он выводит строки только для чтения. Чтобы понять, что это значит, давайте попробуем сохранить результат в список `lines` с помощью обычного присваивания:

In [3]:
lines = f.readlines()

In [4]:
# ха-ха
lines

Список оказался пустым! Более того, если мы снова попробуем вызвать `.readlines()`, мы ничего хорошего не получим:

In [5]:
f.readlines() # все сломалось

Чтобы избежать таких неприятных сюрпризов, лучше воспользоваться циклом, и в цикле заполнить новый список строк.

In [6]:
# дубль два
f = open('mytext.txt', 'r', encoding = 'UTF-8')

lines = []
for l in f.readlines():
    lines.append(l)
    
lines

Теперь все в порядке. Приведем в порядок наши строки. Видно, что в списке строк встречаются «пустые» строки, состоящие из одного символа для перехода на новую строку (`\n`). Кроме того, этот символ встречается на конце строк. Исправим это, используя списковые включения!  

In [7]:
# убираем пустые строки
clean = [l for l in lines if l != '\n']
clean

In [8]:
# убираем \n на конце строк (и лишние пробелы по краям вообще)
clean = [s.strip() for s in clean]
clean

Так как при анализе текстов часто используется модель "мешка слов" (*bag of words*), грамматическая структура предложений, порядок слов и знаки препинания не играют никакой роли. Давайте для начала избавимся от знаков пунктуации. Импортируем модуль `string`, который позволит получить готовую строку со знаками препинания:

In [9]:
import string
string.punctuation

Получили строку, в которой учтены почти все знаки препинания. Почему почти? Так как многие модули ориентированы на англоязычный текст (и вообще текст на латинице), русская пунктуация в рассмотрение не входит. Так, здесь не хватает кавычек-ёлочек, принятых в русскоязычных текстах. Кроме того, здесь не хватает тире. Добавим их. Так как результат `string.punctuation` – это обычная строка, к ней можно добавить свои символы с помощью конкатенации:

In [10]:
to_remove = string.punctuation + '«»—'
to_remove

Убирать из текстов символы, которые есть в строке `to_remove`, можно по-разному. Мы воспользуемся такой хитростью: создадим `translator`, который будет заменять знаки препинания из `to_remove` на пустые строки `''`, а затем будем использовать его в качестве функции, которая будет применяться в методе `translate` для строк.

In [11]:
# создаем translator
translator = str.maketrans('', '', to_remove)

In [12]:
# применяем (на примере одной строки)
s = 'После всех попыток добиться истины, оформитель слышит от неё только: «Забудьте об Анне. Её больше нет».'
s.translate(translator)

**Задание.** Написать функцию `normalize(x)`, которая удаляет в строке `x` все знаки препинания, приводит весь текст к нижнему регистру и возвращает новую строку. 

*Решение:*

In [13]:
def normalize(x):
    to_remove = string.punctuation + '«»—'
    translator = str.maketrans('', '', to_remove)
    res = x.translate(translator)
    res = res.lower()
    return res

Применим функцию к элементам списка `сlean` и назовем новый список `normalized`.

In [14]:
normalized = [normalize(c) for c in clean]
normalized

In [15]:
text = " ".join(normalized)
text

'действие происходит в петербурге в 19081914 годах знаменитый художникоформитель платон андреевич хочет продлить жизнь человека в скульптуре и на рисунках пытаясь побороть смерть и усовершенствовать окружающий мир с помощью своего таланта многие годы ему не давала покоя мысль о состязании с всевышним ему автору великолепных восковых манекенов хотелось создать нечто совершенное и вечное не поддающееся течению времени в 1908 году художник получает заказ от ювелира на оформление витрины магазина в поисках натурщицы для изготовления манекена для витрины художник находит анну молодую девушку смертельно больную чахоткой и ваяет с неё свой лучший манекен вкладывая в работу всю душу проходит время на дворе 1914 год известный ранее художник впал в забвение дела идут совсем не так хорошо как в прежние времена под влиянием творческого кризиса художник начинает злоупотреблять морфием ему грозит полное разорение однажды находясь в крайней нужде платон андреевич принял предложение некоего богатого д

Продолжим выполнять предварительную обработку текста. Прежде чем анализировать текст, смотреть какие слова в нем встречаются чаще и прочее, необходимо максимально унифицировать текст. Нужно сделать так, чтобы все грамматические формы слова считались как одно и то же слово. Например, чтобы слова *страны* и *страной* воспринимались как разные формы одного и того же слова *страна*. Для этого есть два пути: стемминг или лемматизация. 

Стемминг – это процесс обработки, в результате которого от слов остаются только их основы. Так, от слова *страны* останется *стран*, от слова *делает* – *дела*. Этот способ достаточно удобен, в библиотеке `nltk` встроены готовые стеммеры для разных языков, но есть две проблемы. Во-первых, стемминг хуже работает в случае морфологически богатых языков, языков, где возможны совершенно разнообразные формы слова (к таким относится русский язык). Во-вторых, если по итогам анализа текста мы захотим построить облака слов, нам придется писать функцию, которая возвращает предобработанным словам их первоначальный вид, так как "обрезаные" слова в облаке будут смотреться неэстетично и неинформативно.

Лемматизация – это приведение слова к его словарной (начальной) форме. В случае существительных это будет форма единственного числа и именительного падежа, в случае прилагательных – форма единственного числа, мужского рода и именительного падежа, и так далее. Например, слово *белую* в результате лемматизации превратится в *белый*, *стоял* – в *стоять*. Как раз лемматизация отлично подойдет для русского языка. 

Для стемминга и лемматизации русскоязычных слов подойдет [библиотека](https://github.com/nlpub/pymystem3) `pymystem3`, которая является питоновской оболочкой для морфологического анализатора Яндекса. 

Импортируем из этой библиотеки класс `Mystem` (пока про классы можно думать как про наборы методов, которые можно написать самостоятельно и применять к объектам определенного типа).

Чтобы установить библиотеку `pymystem3`, нужно:
* на Windows: в меню Пуск выбрать папку Anaconda3 и Anaconda Command Prompt
* на Mac OS, Linux: открыть новое окно терминала (командной строки)

В Anaconda Command Prompt/терминале ввести

    pip install pymystem3
    
и нажать *Enter*. При подключении к Интернету будут скачаны установочные файлы, а затем компоненты библиотеки будут установлены. В процессе установки могут всплывать предупреждения, но на них можно не обращать внимание, если в итоге есть строка с `successfully installed`.

In [16]:
from pymystem3 import Mystem

In [17]:
m = Mystem()
m

Получим список лемм (слов, приведенных к начальной форме) на основе нашего текста:

In [18]:
lemmas = m.lemmatize(text)
lemmas[0:21] # первые несколько элементов

['действие',
 ' ',
 'происходить',
 ' ',
 'в',
 ' ',
 'петербург',
 ' ',
 'в',
 ' ',
 '19081914',
 ' ',
 'год',
 ' ',
 'знаменитый',
 ' ',
 'художникоформитель',
 ' ',
 'платон',
 ' ',
 'андреевич']

Теперь можем склеить из этого списка слов новый текст и сохранить его в файл на всякий случай.

In [19]:
lemmatized = "".join(m.lemmatize(text))
lemmatized

'действие происходить в петербург в 19081914 год знаменитый художникоформитель платон андреевич хотеть продлять жизнь человек в скульптура и на рисунок пытаться побороть смерть и усовершенствовать окружающий мир с помощь свой талант многий год он не давать покой мысль о состязание с всевышний он автор великолепный восковой манекен хотеться создавать нечто совершать и вечный не поддаваться течение время в 1908 год художник получать заказ от ювелир на оформление витрина магазин в поиск натурщица для изготовление манекен для витрина художник находить анна молодой девушка смертельно больной чахотка и ваять с она свой хороший манекен вкладывать в работа весь душа проходить время на двор 1914 год известный ранее художник впадать в забвение дело идти совсем не так хорошо как в прежний время под влияние творческий кризис художник начинать злоупотреблять морфий он грозить полный разорение однажды находиться в крайний нужда платон андреевич принимать предложение некий богатый делец грильо оформл

In [20]:
lemmatized = lemmatized.rstrip() # уберем \n в конце

Запишем новый файл. Воспользуемся той же функцией `open()`, только выставим флаг `w` (от *write*):

In [21]:
new_f = open('my_text_lemmas.txt', 'w')

In [22]:
print(lemmatized, file = new_f) # впечатаем в файл текст
new_f.close() # и закроем файл

Строка с `.close()` небоходима, поскольку она закрывает файл, сохраняя его таким образом от последующих случайных изменений. Если забыть закрыть файл, то его содержимое может стереться при следующем обращении.

С помощью класса `Mystem` можно делать и более интересные вещи. Например, узнавать части речи слов и их грамматические признаки:

In [23]:
# m.analyze(text)
m.analyze(text)[0:11]

[{'analysis': [{'lex': 'действие',
    'wt': 1,
    'gr': 'S,сред,неод=(вин,ед|им,ед)'}],
  'text': 'действие'},
 {'text': ' '},
 {'analysis': [{'lex': 'происходить',
    'wt': 1,
    'gr': 'V,нп=непрош,ед,изъяв,3-л,несов'}],
  'text': 'происходит'},
 {'text': ' '},
 {'analysis': [{'lex': 'в', 'wt': 0.9999917878, 'gr': 'PR='}], 'text': 'в'},
 {'text': ' '},
 {'analysis': [{'lex': 'петербург', 'wt': 1, 'gr': 'S,гео,муж,неод=пр,ед'}],
  'text': 'петербурге'},
 {'text': ' '},
 {'analysis': [{'lex': 'в', 'wt': 0.9999917878, 'gr': 'PR='}], 'text': 'в'},
 {'text': ' '},
 {'text': '19081914'}]

Здесь `S` – это существительное, `V` – глагол, `PR` – предлог. Сокращением `lex` обозначена лексема – словарная форма слова, а `text` – это слово в том виде, в котором оно встречается в тексте. Через `|` обозначены спорные формы. Например, по одному слову непонятно, именительный это падеж, или винительный.

**Бонус**

На самом деле, остался еще один этап предобработки текста – удаление стоп-слов. Стоп-слова – это часто встречающиеся слова, которые не несут важной смысловой информации. К таким словам обычно относят предлоги, частицы, союзы, некоторые наречия, междометия. Ведь, если частотный анализ текстов покажет нам, например, что для одной группы документов характерны слова «для» и «более», а для другой ‒ «не» и «что», это вряд ли позволит нам сделать осмысленные содержательные выводы. Базовые списки стоп-слов, встроенные в различные библиотеки, обычно расширяют, добавляя новые слова, исходя из контекста. Так, если мы работаем с постами пользователей, и в каждом посте обнаруживаем сочетание вида «имя пользователя», слова *имя* и *пользователь* имеет смысл отнести к стоп-словам, потому что они в любом случае будут среди самых распространенных, вне зависимости от тематики поста. 

Установим библиотеку `nltk` (*Natural Language Toolkit*). Установить библиотеку можно точно так же, как и `mystem3` через:

    pip install nltk

Но для работы с отдельными модулями библиотеки нужно их доставить прямо через Jupyter. Нам понадобятся модули для токенизации (разбиение текста на слова) и для выбора стоп-слов. Если прогнать следующий код, появится отдельное окно (вне Jupyter) и в нем можно будет выбрать все модули (*all*, кликнуть внизу *Download* и  подождать):

In [24]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [25]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

stopWords = set(stopwords.words('russian')) # для русского языка

In [26]:
# stopWords # можно раскомментировать и посмотреть на множество слов (оно довольно объемное)

In [27]:
words = word_tokenize(lemmatized) # разбиваем текст на слова
wordsFiltered = []

for w in words:
    if w not in stopWords: # фильтруем слова - все, что вне множества StopWords
        wordsFiltered.append(w)

print(wordsFiltered)

['действие', 'происходить', 'петербург', '19081914', 'год', 'знаменитый', 'художникоформитель', 'платон', 'андреевич', 'хотеть', 'продлять', 'жизнь', 'человек', 'скульптура', 'рисунок', 'пытаться', 'побороть', 'смерть', 'усовершенствовать', 'окружающий', 'мир', 'помощь', 'свой', 'талант', 'многий', 'год', 'давать', 'покой', 'мысль', 'состязание', 'всевышний', 'автор', 'великолепный', 'восковой', 'манекен', 'хотеться', 'создавать', 'нечто', 'совершать', 'вечный', 'поддаваться', 'течение', 'время', '1908', 'год', 'художник', 'получать', 'заказ', 'ювелир', 'оформление', 'витрина', 'магазин', 'поиск', 'натурщица', 'изготовление', 'манекен', 'витрина', 'художник', 'находить', 'анна', 'молодой', 'девушка', 'смертельно', 'больной', 'чахотка', 'ваять', 'свой', 'хороший', 'манекен', 'вкладывать', 'работа', 'весь', 'душа', 'проходить', 'время', 'двор', '1914', 'год', 'известный', 'ранее', 'художник', 'впадать', 'забвение', 'дело', 'идти', 'прежний', 'время', 'влияние', 'творческий', 'кризис', 'х

На этом с предварительной обработкой текста можно закончить. Мы привели все к нижнему регистру, удалили пунктуацию, провели лемматизацию, превратили текст в список слов, убрав стоп-слова. По-хорошему, нужно было ещё убрать числа, но мы их пока оставили.